In [111]:

!git add PredictNHL.ipynb


!git commit -m "Add PredictNHL.ipynb"


!git pull origin main

!git push origin main

[main cd0f522] Add PredictNHL.ipynb
 1 file changed, 2366 insertions(+)
 create mode 100644 Desktop/PredictNHL.ipynb
Merge made by the 'ort' strategy.
 README.md | 72 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++------
 1 file changed, 66 insertions(+), 6 deletions(-)


From https://github.com/nic01as1/NHLPredictor
 * branch            main       -> FETCH_HEAD
   bd65cfb..58035ac  main       -> origin/main
To https://github.com/nic01as1/NHLPredictor.git
   58035ac..384e0f7  main -> main


In [ ]:
#if stuck :
!git push origin main --force


In [1]:
!pip install tensorflow
!pip install xgboost
!pip install streamlit
!pip install imbalanced-learn
!pip install shap
!pip install statsmodels
!pip install lightgbm


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import time
import json
import random
import warnings
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, TimeSeriesSplit, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import f1_score, roc_auc_score, brier_score_loss, accuracy_score, log_loss, classification_report, make_scorer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_classif, RFECV, SelectFromModel, SelectKBest
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
import joblib


In [2]:
#Retrieve the rank of each teams

url = 'https://www.hockey-reference.com/leagues/NHL_2025.html#all_stats'

with webdriver.Chrome() as driver:
    driver.get(url)
    time.sleep(2) 
    soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.quit()

table = soup.find('table', id='stats')

if table is None:
    print("Team Statistics table not found")
else:
    
    rows = table.find_all('tr')[1:] 
    teams = [
        row.find_all('td')[0].text.strip()
        for row in rows
        if row.find_all('td')
    ]
    ranks = list(range(1, len(teams) + 1)) 

    
    df_rank = pd.DataFrame({
        'Team': teams,
        'Rank': ranks,
        'Team Rank': [f"{team} {rank}" for team, rank in zip(teams, ranks)]
    })
    df_rank = df_rank[df_rank['Team'] != 'League Average'].reset_index(drop=True)
    df_rank['Team'] = df_rank['Team'].str.replace('*', '', regex=False).str.strip()

print(df_rank)

    

                     Team  Rank                 Team Rank
0           Winnipeg Jets     1          Winnipeg Jets* 1
1     Washington Capitals     2    Washington Capitals* 2
2    Vegas Golden Knights     3   Vegas Golden Knights* 3
3     Toronto Maple Leafs     4    Toronto Maple Leafs* 4
4            Dallas Stars     5           Dallas Stars* 5
5       Los Angeles Kings     6      Los Angeles Kings* 6
6     Tampa Bay Lightning     7    Tampa Bay Lightning* 7
7      Colorado Avalanche     8     Colorado Avalanche* 8
8         Edmonton Oilers     9        Edmonton Oilers* 9
9     Carolina Hurricanes    10   Carolina Hurricanes* 10
10       Florida Panthers    11      Florida Panthers* 11
11         Minnesota Wild    12        Minnesota Wild* 12
12        Ottawa Senators    13       Ottawa Senators* 13
13         Calgary Flames    14         Calgary Flames 14
14        St. Louis Blues    15       St. Louis Blues* 15
15     Montreal Canadiens    16    Montreal Canadiens* 16
16      New Je

In [3]:

#Retrieve the infos of EACH games for the season

def scrape_game_history(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table")
    if table is None:
        print(f"Table not found on {url}")
        return pd.DataFrame()  #

    
    data = {
        "Date": [],
        "Home Team": [],
        "Home Score": [],
        "Away Team": [],
        "Away Score": [],
        "Result": [],
        "Home Win": []
    }

    
    for row in table.find_all("tr")[1:]:  # Skip the header
        columns = row.find_all("td")
        if len(columns) < 6:  
            continue

        
        date = columns[1].text.strip()
        home_team = columns[3].text.strip()
        away_team = columns[4].text.strip()
        result = columns[5].text.strip()

        
        if "-" in result:  
            scores = result.split("-")
            if len(scores) == 2 and scores[0].strip().isdigit() and scores[1].strip().isdigit():
                home_score = int(scores[0].strip())
                away_score = int(scores[1].strip())
                home_win = home_score > away_score
            else:  
                home_score = away_score = home_win = None
        else:
            home_score = away_score = home_win = None

        
        data["Date"].append(date)
        data["Home Team"].append(home_team)
        data["Away Team"].append(away_team)
        data["Result"].append(result)
        data["Home Score"].append(home_score)
        data["Away Score"].append(away_score)
        data["Home Win"].append(home_win)

    
    df_game = pd.DataFrame(data)
    df_game["Date"] = df_game["Date"].apply(lambda x: x.split(" ")[0])  # Extract just the date
    return df_game



url_history = "https://fixturedownload.com/results/nhl-2024"


df_game = scrape_game_history(url_history)


if df_game.empty:
    print("No game data found.")
else:
    print(f"Scraped {len(df_game)} games.")
    print(df_game)

Scraped 1312 games.
            Date              Home Team  Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres           1    New Jersey Devils   
1     05/10/2024      New Jersey Devils           3       Buffalo Sabres   
2     08/10/2024         Seattle Kraken           2      St. Louis Blues   
3     08/10/2024       Florida Panthers           6        Boston Bruins   
4     09/10/2024       Utah Hockey Club           5   Chicago Blackhawks   
...          ...                    ...         ...                  ...   
1307  17/04/2025        Ottawa Senators           7  Carolina Hurricanes   
1308  17/04/2025       New York Rangers           4  Tampa Bay Lightning   
1309  17/04/2025    Pittsburgh Penguins           5  Washington Capitals   
1310  17/04/2025  Columbus Blue Jackets           6   New York Islanders   
1311  18/04/2025      Los Angeles Kings           1       Calgary Flames   

      Away Score Result  Home Win  
0              4  1 - 4     Fal

In [4]:
#Normalise Both datasource
team_name_mapping = {
    "Montréal Canadiens": "Montreal Canadiens", 
    
}


df_game["Home Team"] = df_game["Home Team"].replace(team_name_mapping)
df_game["Away Team"] = df_game["Away Team"].replace(team_name_mapping)


In [5]:
# Merge df_rank to add Home Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Home Team", "Rank": "Home Team Rank"}), 
    on="Home Team", 
    how="left"
)

# Merge df_rank to add Away Team Rank
df_game = df_game.merge(
    df_rank.rename(columns={"Team": "Away Team", "Rank": "Away Team Rank"}), 
    on="Away Team", 
    how="left"
)

# Reorder the final columns
final_columns = [
    "Date", "Home Team", "Home Score", "Away Team", "Away Score",
    "Home Team Rank", "Away Team Rank", "Home Win"
]
df_final = df_game.loc[:, final_columns] 

print(df_final)


            Date              Home Team  Home Score            Away Team  \
0     04/10/2024         Buffalo Sabres           1    New Jersey Devils   
1     05/10/2024      New Jersey Devils           3       Buffalo Sabres   
2     08/10/2024         Seattle Kraken           2      St. Louis Blues   
3     08/10/2024       Florida Panthers           6        Boston Bruins   
4     09/10/2024       Utah Hockey Club           5   Chicago Blackhawks   
...          ...                    ...         ...                  ...   
1307  17/04/2025        Ottawa Senators           7  Carolina Hurricanes   
1308  17/04/2025       New York Rangers           4  Tampa Bay Lightning   
1309  17/04/2025    Pittsburgh Penguins           5  Washington Capitals   
1310  17/04/2025  Columbus Blue Jackets           6   New York Islanders   
1311  18/04/2025      Los Angeles Kings           1       Calgary Flames   

      Away Score  Home Team Rank  Away Team Rank  Home Win  
0              4          

In [6]:
#DATA MANIPULATION


# Filter out the Unplayed Games
df_final = df_final.dropna(subset=["Home Score", "Away Score"])

# Convert Scores and Dates to Appropriate Data Types
df_final["Date"] = pd.to_datetime(df_final["Date"], format="%d/%m/%Y")
df_final[["Home Score", "Away Score"]] = df_final[["Home Score", "Away Score"]].astype(int)
df_final["Home Win"] = df_final["Home Win"].astype(bool)

# Sort Data by Date for Chronological Calculations
df_final = df_final.sort_values(by="Date").reset_index(drop=True)

# Function: Calculate Last 10 Games Stats
def calculate_last_10_stats(df, team_column):
    last_10_wins = []
    team_games = {team: [] for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        recent_games = team_games[team][-10:]  # Last 10 games
        last_10_wins.append(sum(recent_games))
        team_games[row["Home Team"]].append(row["Home Win"])
        team_games[row["Away Team"]].append(not row["Home Win"])
    return last_10_wins

# Add Last 10 Wins for Home and Away Teams
df_final["Home Last 10 Wins"] = calculate_last_10_stats(df_final, "Home Team")
df_final["Away Last 10 Wins"] = calculate_last_10_stats(df_final, "Away Team")

# Add Whether Teams Played Yesterday
def calculate_played_yesterday(df, team_column):
    played_yesterday = []
    last_game_date = {}

    for _, row in df.iterrows():
        team = row[team_column]
        played_yesterday.append(last_game_date.get(team) == row["Date"] - pd.Timedelta(days=1))
        last_game_date[team] = row["Date"]
    return played_yesterday

df_final["Home Played Yesterday"] = calculate_played_yesterday(df_final, "Home Team")
df_final["Away Played Yesterday"] = calculate_played_yesterday(df_final, "Away Team")

# Add Win Rate for Home and Away Teams
def calculate_win_rate(df, team_column, is_home_column):
    win_rate = []
    team_stats = {team: {"wins": 0, "games": 0} for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        stats = team_stats[team]
        win_rate.append(stats["wins"] / stats["games"] if stats["games"] > 0 else 0)
        
        
        if is_home_column:
            stats["wins"] += row["Home Win"]
        else:
            stats["wins"] += not row["Home Win"]
        stats["games"] += 1
    return win_rate

df_final["Home Win Rate"] = calculate_win_rate(df_final, "Home Team", is_home_column=True)
df_final["Away Win Rate"] = calculate_win_rate(df_final, "Away Team", is_home_column=False)

# Add Overall Win Streak for Home and Away Teams
def calculate_overall_win_streak(df):
    streak = {}
    home_streaks = []
    away_streaks = []

    for _, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]

        
        home_streaks.append(streak.get(home_team, 0))
        away_streaks.append(streak.get(away_team, 0))

        
        if row["Home Win"]:
            streak[home_team] = streak.get(home_team, 0) + 1
            streak[away_team] = 0
        else:
            streak[home_team] = 0
            streak[away_team] = streak.get(away_team, 0) + 1
    return home_streaks, away_streaks

home_streaks, away_streaks = calculate_overall_win_streak(df_final)
df_final["Home Team Overall Win Streak Before Game"] = home_streaks
df_final["Away Team Overall Win Streak Before Game"] = away_streaks

# Add Average Opponent Strength for Home and Away Teams
def calculate_avg_opponent_rank(df, team_column, opponent_rank_column):
    avg_opponent_rank = []
    opponent_stats = {team: [] for team in pd.concat([df["Home Team"], df["Away Team"]]).unique()}

    for _, row in df.iterrows():
        team = row[team_column]
        avg_opponent_rank.append(
            np.mean(opponent_stats[team][-10:]) if opponent_stats[team] else np.nan
        )
        opponent_stats[row["Home Team"]].append(row["Away Team Rank"])
        opponent_stats[row["Away Team"]].append(row["Home Team Rank"])
    return avg_opponent_rank

df_final["Home Opponent Strength"] = calculate_avg_opponent_rank(df_final, "Home Team", "Away Team Rank")
df_final["Away Opponent Strength"] = calculate_avg_opponent_rank(df_final, "Away Team", "Home Team Rank")

def calculate_days_since_last_game(df):
    last_game_date = {}  # Dictionary to track the last game date for each team
    home_days_since_last_game = []
    away_days_since_last_game = []

    for _, row in df.iterrows():
        home_team = row["Home Team"]
        away_team = row["Away Team"]
        current_date = row["Date"]

        # Calculate days since last game for the home team
        if home_team in last_game_date:
            days_since_last_home_game = (current_date - last_game_date[home_team]).days
        else:
            days_since_last_home_game = None  
        home_days_since_last_game.append(days_since_last_home_game)

        # Calculate days since last game for the away team
        if away_team in last_game_date:
            days_since_last_away_game = (current_date - last_game_date[away_team]).days
        else:
            days_since_last_away_game = None  
        away_days_since_last_game.append(days_since_last_away_game)

        # Update the last game date for both teams
        last_game_date[home_team] = current_date
        last_game_date[away_team] = current_date

    return home_days_since_last_game, away_days_since_last_game


# Apply the function to the DataFrame
home_days, away_days = calculate_days_since_last_game(df_final)


df_final["Home Rest Days Since Last Game"] = home_days
df_final["Away Rest Days Since Last Game"] = away_days

print(df_final[["Date", "Home Team", "Away Team", "Home Opponent Strength", "Away Opponent Strength","Home Last 10 Wins","Away Last 10 Wins","Home Played Yesterday","Away Played Yesterday","Home Win Rate","Away Win Rate","Home Team Overall Win Streak Before Game","Away Team Overall Win Streak Before Game","Away Rest Days Since Last Game","Home Rest Days Since Last Game"]].tail(10))


df_final.to_csv("enhanced_game_data.csv", index=False)


           Date              Home Team             Away Team  \
1302 2025-04-17        Ottawa Senators   Carolina Hurricanes   
1303 2025-04-17    Pittsburgh Penguins   Washington Capitals   
1304 2025-04-17       New York Rangers   Tampa Bay Lightning   
1305 2025-04-17    Toronto Maple Leafs     Detroit Red Wings   
1306 2025-04-17  Columbus Blue Jackets    New York Islanders   
1307 2025-04-17        San Jose Sharks       Edmonton Oilers   
1308 2025-04-17      Vancouver Canucks  Vegas Golden Knights   
1309 2025-04-17    Nashville Predators          Dallas Stars   
1310 2025-04-17         Buffalo Sabres   Philadelphia Flyers   
1311 2025-04-18      Los Angeles Kings        Calgary Flames   

      Home Opponent Strength  Away Opponent Strength  Home Last 10 Wins  \
1302                    20.1                    15.9                  6   
1303                    18.3                    20.0                  4   
1304                    17.2                    18.9                  

In [7]:
df_final["Rank Difference"] = df_final["Home Team Rank"] - df_final["Away Team Rank"]
df_final["Home Advantage"] = df_final["Home Win Rate"] - df_final["Away Win Rate"]
df_final["Win Streak Impact"] = df_final["Home Team Overall Win Streak Before Game"] - df_final["Away Team Overall Win Streak Before Game"]
df_final["Opponent Strength"] = df_final["Home Opponent Strength"] - df_final["Away Opponent Strength"]
df_final["Last 10 Wins"] = df_final["Home Last 10 Wins"] - df_final["Away Last 10 Wins"]
df_final["Opponent Strength"] = df_final["Home Opponent Strength"] - df_final["Away Opponent Strength"]

In [8]:
df_final = df_final.dropna().reset_index(drop=True)

results1 = df_final
results1.tail(10)

,Date,Home Team,Home Score,Away Team,Away Score,Home Team Rank,Away Team Rank,Home Win,Home Last 10 Wins,Away Last 10 Wins,...,Away Team Overall Win Streak Before Game,Home Opponent Strength,Away Opponent Strength,Home Rest Days Since Last Game,Away Rest Days Since Last Game,Rank Difference,Home Advantage,Win Streak Impact,Opponent Strength,Last 10 Wins
1282,2025-04-17,Ottawa Senators,7,Carolina Hurricanes,5,13,10,True,6,4,...,0,20.1,15.9,2.0,1.0,3,0.250,0,4.2,2
1283,2025-04-17,Pittsburgh Penguins,5,Washington Capitals,2,24,2,True,4,4,...,1,18.3,20.0,4.0,2.0,22,-0.125,-1,-1.7,0
1284,2025-04-17,New York Rangers,4,Tampa Bay Lightning,0,22,7,True,4,6,...,2,17.2,18.9,3.0,2.0,15,0.000,-1,-1.7,-2
1285,2025-04-17,Toronto Maple Leafs,4,Detroit Red Wings,3,4,21,True,8,6,...,3,16.3,13.2,2.0,1.0,-17,0.225,1,3.1,2
1286,2025-04-17,Columbus Blue Jackets,6,New York Islanders,1,19,23,True,6,3,...,0,14.0,13.8,2.0,2.0,-4,0.225,5,0.2,3
1287,2025-04-17,San Jose Sharks,0,Edmonton Oilers,3,32,9,False,0,6,...,0,15.7,16.2,2.0,2.0,23,-0.250,0,-0.5,-6
1288,2025-04-17,Vancouver Canucks,1,Vegas Golden Knights,4,18,3,False,5,6,...,0,15.6,18.3,2.0,1.0,15,-0.075,1,-2.7,-1
1289,2025-04-17,Nashville Predators,5,Dallas Stars,1,30,5,True,2,4,...,0,15.3,19.6,2.0,3.0,25,-0.075,0,-4.3,-2
1290,2025-04-17,Buffalo Sabres,5,Philadelphia Flyers,4,26,29,True,5,5,...,0,12.9,20.0,2.0,2.0,-3,0.225,0,-7.1,0
1291,2025-04-18,Los Angeles Kings,1,Calgary Flames,5,6,14,False,8,6,...,3,16.4,16.9,2.0,2.0,-8,0.350,1,-0.5,2


In [9]:

scorer = make_scorer(f1_score, pos_label=None, average='weighted')

y_true = df_final["Home Win"].astype(int).values

y_pred = np.ones_like(y_true)                   
y_proba = np.ones_like(y_true, dtype=float)     

f1 = f1_score(y_true, y_pred, average="weighted")


try:
    roc = roc_auc_score(y_true, y_proba)
except ValueError:
    roc = float("nan")  # undefined for constant scores


brier = brier_score_loss(y_true, y_proba)

print(f"Baseline (Always Home) — F1: {f1:.4f}, ROC AUC: {roc}, Brier: {brier:.4f}")
y_pred_away = np.zeros_like(y_true)
y_proba_away = np.zeros_like(y_true, dtype=float)

f1_away = f1_score(y_true, y_pred_away, average="weighted")
try:
    roc_away = roc_auc_score(y_true, y_proba_away)
except ValueError:
    roc_away = float("nan")
brier_away = brier_score_loss(y_true, y_proba_away)

print(f"Baseline (Always Away) — F1: {f1_away:.4f}, ROC AUC: {roc_away}, Brier: {brier_away:.4f}")

Baseline (Always Home) — F1: 0.4098, ROC AUC: 0.5, Brier: 0.4334
Baseline (Always Away) — F1: 0.2621, ROC AUC: 0.5, Brier: 0.5666


In [27]:
df_sorted = df_final.sort_values("Date").reset_index(drop=True)

feature_cols = [
    "Opponent Strength","Rank Difference","Last 10 Wins",
    "Home Played Yesterday","Away Played Yesterday",
    "Home Advantage","Win Streak Impact",
    "Away Rest Days Since Last Game","Home Rest Days Since Last Game"
]
target_col = "Home Win"

X_all = df_sorted[feature_cols]
y_all = df_sorted[target_col].astype(int)

split_idx = int(0.8 * len(df_sorted))
X_train, X_test = X_all.iloc[:split_idx], X_all.iloc[split_idx:]
y_train, y_test = y_all.iloc[:split_idx], y_all.iloc[split_idx:]


In [28]:

def compute_vif(df):
    import statsmodels.api as sm
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    X = sm.add_constant(df)
    vif = pd.Series(
        [variance_inflation_factor(X.values, i) for i in range(1, X.shape[1])],
        index=df.columns
    )
    return vif.sort_values(ascending=False)

vif_series = compute_vif(X_train.select_dtypes(include=[np.number]))
print("VIF:\n", vif_series)



VIF:
 Last 10 Wins                      1.767266
Away Rest Days Since Last Game    1.534972
Home Rest Days Since Last Game    1.529276
Rank Difference                   1.527904
Home Advantage                    1.432449
Win Streak Impact                 1.302627
Opponent Strength                 1.035650
dtype: float64


In [29]:
# 0) Features & split
feature_cols = [
    "Opponent Strength","Rank Difference","Last 10 Wins",
    "Home Played Yesterday","Away Played Yesterday",
    "Home Advantage","Win Streak Impact",
    "Away Rest Days Since Last Game","Home Rest Days Since Last Game"
]
target_col = "Home Win"

df = df_final.sort_values("Date").reset_index(drop=True)
rename_map = {c: c.replace(" ", "_") for c in feature_cols}
df = df.rename(columns=rename_map)

feat = [rename_map[c] for c in feature_cols]
X, y = df[feat], df[target_col].astype(int)

split = int(0.8 * len(df))
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

cv5 = StratifiedKFold(n_splits=5, shuffle=False)

# 1) Name-preserving selectors
class NamePreservingSelectFromModel(TransformerMixin, BaseEstimator):
    def __init__(self, estimator, threshold="median"):
        self.estimator = estimator
        self.threshold = threshold
        self.selected_cols_ = None
    def fit(self, X, y=None):
        if not hasattr(X, "columns"): raise TypeError("Need pandas DataFrame.")
        est = clone(self.estimator).fit(X, y)
        sfm = SelectFromModel(est, threshold=self.threshold, prefit=True)
        self.selected_cols_ = list(X.columns[sfm.get_support()])
        return self
    def transform(self, X): return X[self.selected_cols_]

class NamePreservingSelectKBest(TransformerMixin, BaseEstimator):
    def __init__(self, score_func=mutual_info_classif, k=5):
        self.score_func = score_func; self.k = k; self.selected_cols_ = None
    def fit(self, X, y=None):
        if not hasattr(X, "columns"): raise TypeError("Need pandas DataFrame.")
        skb = SelectKBest(self.score_func, k=self.k).fit(X, y)
        self.selected_cols_ = list(X.columns[skb.get_support()])
        return self
    def transform(self, X): return X[self.selected_cols_]

class NamePreservingRFECV(TransformerMixin, BaseEstimator):
    def __init__(self, estimator, step=1, cv=None, scoring="neg_brier_score", min_features_to_select=1):
        self.estimator = estimator; self.step = step; self.cv = cv
        self.scoring = scoring; self.min_features_to_select = min_features_to_select
        self.selected_cols_ = None
    def fit(self, X, y=None):
        if not hasattr(X, "columns"): raise TypeError("Need pandas DataFrame.")
        r = RFECV(self.estimator, step=self.step, cv=self.cv, scoring=self.scoring,
                  min_features_to_select=self.min_features_to_select).fit(X, y)
        self.selected_cols_ = list(X.columns[r.support_]); return self
    def transform(self, X): return X[self.selected_cols_]

# 2) Pipelines
pipe_logistic = Pipeline([
    ("rfe", NamePreservingRFECV(LogisticRegression(max_iter=1000), step=1, cv=cv5, scoring="neg_brier_score")),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])

pipe_mlp = Pipeline([
    ("skb", NamePreservingSelectKBest(k=max(3, len(feat)//2))),
    ("scaler", StandardScaler()),
    ("clf", MLPClassifier(max_iter=500, random_state=42))
])

pipe_lgbm = Pipeline([
    ("sfm", NamePreservingSelectFromModel(LGBMClassifier(random_state=42), threshold="median")),
    ("clf", LGBMClassifier(random_state=42))
])

pipe_rf = Pipeline([
    ("sfm", NamePreservingSelectFromModel(RandomForestClassifier(n_estimators=400, random_state=42), threshold="median")),
    ("clf", RandomForestClassifier(n_estimators=400, random_state=42))
])

candidates = {
    "Logistic": pipe_logistic,
    "MLP": pipe_mlp,
    "RandomForest": pipe_rf,
    "LightGBM": pipe_lgbm,
}

# 3) Fit / evaluate / save
def evaluate(model, X_te, y_te):
    y_hat = model.predict(X_te)
    p = model.predict_proba(X_te)[:, 1]
    return {"F1": f1_score(y_te, y_hat, average="weighted"),
            "ROC_AUC": roc_auc_score(y_te, p),
            "Brier": brier_score_loss(y_te, p)}

results, selected = [], {}

for name, model in candidates.items():
    model.fit(X_train, y_train)
    results.append({"Model": name, **evaluate(model, X_test, y_test)})
    # grab selected cols from whichever selector exists
    sel_cols = None
    for key in ("rfe", "skb", "sfm"):
        if key in model.named_steps and hasattr(model.named_steps[key], "selected_cols_"):
            sel_cols = model.named_steps[key].selected_cols_
    selected[name] = sel_cols
    joblib.dump(model, f"model_{name}.joblib")

with open("selected_features.json", "w") as f:
    json.dump({"rename_map": rename_map, "selected_features": selected}, f, indent=2)

res = (pd.DataFrame(results)
         .sort_values(["Brier","ROC_AUC","F1"], ascending=[True, False, False])
         .reset_index(drop=True))
print("\n=== Models ===\n", res[["Model","F1","ROC_AUC","Brier"]])
print("\nSelected features:\n", selected)

# 4) Best by Brier; calibrate if not Logistic
best = res.iloc[0]["Model"]
pipe = joblib.load(f"model_{best}.joblib")

final_model = pipe if best == "Logistic" else CalibratedClassifierCV(pipe, method="isotonic", cv=5).fit(X_train, y_train)
joblib.dump(final_model, f"model_{best}_CALIBRATED.joblib")

print("\nBest:", best)
print("Final metrics:", evaluate(final_model, X_test, y_test))

# 5) Load & predict helper
def load_model_and_predict(model_name: str, X_new: pd.DataFrame) -> np.ndarray:
    with open("selected_features.json", "r") as f:
        meta = json.load(f)
    Xn = X_new.rename(columns={k: v for k, v in meta["rename_map"].items() if k in X_new.columns})
    path = f"model_{model_name}_CALIBRATED.joblib"
    if not os.path.exists(path): path = f"model_{model_name}.joblib"
    mdl = joblib.load(path)
    return mdl.predict_proba(Xn)[:, 1]


[LightGBM] [Info] Number of positive: 576, number of negative: 457
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000153 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 625
[LightGBM] [Info] Number of data points in the train set: 1033, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.557599 -> initscore=0.231424
[LightGBM] [Info] Start training from score 0.231424
[LightGBM] [Info] Number of positive: 576, number of negative: 457
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000067 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 601
[LightGBM] [Info] Number of data points in the train set: 1033, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.557599 -> initscore=0.231424
[LightGBM] [Info] Start training from score 0.231424

=== Models ===
           Mod

In [30]:

SEED=42
CV_SPLITS=6
N_ITER_LOG_L2=40
N_ITER_LOG_EN=40
N_ITER_MLP=60
RFE_MIN_FEATS=1
MLP_MAX_ITER=1000
MLP_EARLY_STOP=True
MLP_VAL_FRAC=0.15
MLP_PATIENCE=15
TEST_RATIO=0.2

FEATURE_COLS=[
    "Opponent Strength","Rank Difference","Last 10 Wins",
    "Home Played Yesterday","Away Played Yesterday",
    "Home Advantage","Win Streak Impact",
    "Away Rest Days Since Last Game","Home Rest Days Since Last Game"
]
TARGET_COL="Home Win"
# seeds
random.seed(SEED); np.random.seed(SEED)

# -------- data (time-ordered split)
df=df_final.sort_values("Date").reset_index(drop=True)
rename_map={c:c.replace(" ","_") for c in FEATURE_COLS}
df=df.rename(columns=rename_map)
feat=[rename_map[c] for c in FEATURE_COLS]
X,y=df[feat],df[TARGET_COL].astype(int)
cut=int((1-TEST_RATIO)*len(df))
X_tr,X_te=X.iloc[:cut],X.iloc[cut:]; y_tr,y_te=y.iloc[:cut],y.iloc[cut:]
tscv=TimeSeriesSplit(n_splits=CV_SPLITS)

# -------- tiny helpers
def evaluate(m,Xt,yt):
    p=m.predict_proba(Xt)[:,1]; yhat=(p>=0.5).astype(int)
    return {"F1":f1_score(yt,yhat,average="weighted"),
            "ROC_AUC":roc_auc_score(yt,p),
            "Brier":brier_score_loss(yt,p)}

# -------- name-preserving selectors
class NPSFM(TransformerMixin,BaseEstimator):
    def __init__(self,estimator,threshold="median"): self.estimator=estimator; self.threshold=threshold; self.selected_cols_=None
    def fit(self,X,y=None):
        if not hasattr(X,"columns"): raise TypeError("Need pandas DataFrame.")
        sfm=SelectFromModel(clone(self.estimator).fit(X,y),threshold=self.threshold,prefit=True)
        self.selected_cols_=list(X.columns[sfm.get_support()]); return self
    def transform(self,X): return X[self.selected_cols_]

class NPSKB(TransformerMixin,BaseEstimator):
    def __init__(self,score_func=mutual_info_classif,k=5): self.score_func=score_func; self.k=k; self.selected_cols_=None
    def fit(self,X,y=None):
        if not hasattr(X,"columns"): raise TypeError("Need pandas DataFrame.")
        skb=SelectKBest(self.score_func,k=self.k).fit(X,y)
        self.selected_cols_=list(X.columns[skb.get_support()]); return self
    def transform(self,X): return X[self.selected_cols_]

class NPRFECV(TransformerMixin,BaseEstimator):
    def __init__(self,estimator,step=1,cv=None,scoring="neg_brier_score",min_features_to_select=1):
        self.estimator=estimator; self.step=step; self.cv=cv; self.scoring=scoring; self.min_features_to_select=min_features_to_select; self.selected_cols_=None
    def fit(self,X,y=None):
        if not hasattr(X,"columns"): raise TypeError("Need pandas DataFrame.")
        r=RFECV(self.estimator,step=self.step,cv=self.cv,scoring=self.scoring,min_features_to_select=self.min_features_to_select).fit(X,y)
        self.selected_cols_=list(X.columns[r.support_]); return self
    def transform(self,X): return X[self.selected_cols_]

# -------- pipelines
pipe_log_l2=Pipeline([
    ("rfe",NPRFECV(LogisticRegression(max_iter=2000,solver="lbfgs"),
                   step=1,cv=tscv,scoring="neg_brier_score",min_features_to_select=RFE_MIN_FEATS)),
    ("scaler",StandardScaler()),
    ("clf",LogisticRegression(max_iter=2000,solver="lbfgs"))
])

pipe_log_en=Pipeline([
    ("rfe",NPRFECV(LogisticRegression(max_iter=2000,solver="saga",penalty="elasticnet",l1_ratio=0.5,random_state=SEED),
                   step=1,cv=tscv,scoring="neg_brier_score",min_features_to_select=RFE_MIN_FEATS)),
    ("scaler",StandardScaler()),
    ("clf",LogisticRegression(max_iter=2000,solver="saga",penalty="elasticnet",random_state=SEED))
])

pipe_mlp=Pipeline([
    ("skb",NPSKB(k=max(4,len(feat)//2))),
    ("scaler",StandardScaler()),
    ("clf",MLPClassifier(max_iter=MLP_MAX_ITER,random_state=SEED,
                         early_stopping=MLP_EARLY_STOP,n_iter_no_change=MLP_PATIENCE,
                         validation_fraction=MLP_VAL_FRAC,learning_rate="adaptive"))
])

# -------- search spaces
param_log_l2={"clf__C":loguniform(1e-3,1e1),"clf__class_weight":[None,"balanced"]}
param_log_en={"clf__C":loguniform(1e-3,1e1),"clf__l1_ratio":uniform(0,1),"clf__class_weight":[None,"balanced"]}
param_mlp={"skb__k":[max(4,len(feat)//2),max(6,len(feat)//2+2)],
           "clf__hidden_layer_sizes":[(32,),(64,),(128,),(64,32),(128,64)],
           "clf__alpha":loguniform(1e-6,1e-2),
           "clf__learning_rate_init":loguniform(1e-3,2e-2),
           "clf__activation":["relu","tanh"],
           "clf__batch_size":[32,64,128]}
scoring={"brier":"neg_brier_score","roc_auc":"roc_auc","f1w":"f1_weighted"}

# -------- tune + train (CV then refit best)
rs_log_l2=RandomizedSearchCV(pipe_log_l2,param_log_l2,n_iter=N_ITER_LOG_L2,scoring=scoring,refit="brier",cv=tscv,n_jobs=-1,verbose=1,random_state=SEED).fit(X_tr,y_tr)
rs_log_en=RandomizedSearchCV(pipe_log_en,param_log_en,n_iter=N_ITER_LOG_EN,scoring=scoring,refit="brier",cv=tscv,n_jobs=-1,verbose=1,random_state=SEED).fit(X_tr,y_tr)
rs_mlp   =RandomizedSearchCV(pipe_mlp,   param_mlp,   n_iter=N_ITER_MLP,   scoring=scoring,refit="brier",cv=tscv,n_jobs=-1,verbose=1,random_state=SEED).fit(X_tr,y_tr)

best_log_l2, best_log_en, best_mlp = rs_log_l2.best_estimator_, rs_log_en.best_estimator_, rs_mlp.best_estimator_

# -------- holdout scores
m_l2, m_en, m_mlp = evaluate(best_log_l2,X_te,y_te), evaluate(best_log_en,X_te,y_te), evaluate(best_mlp,X_te,y_te)
print("L2:",m_l2,"\nEN:",m_en,"\nMLP:",m_mlp)

# -------- choose logistic winner; calibrate MLP if it helps Brier
log_final, log_metrics = (best_log_l2,m_l2) if m_l2["Brier"]<=m_en["Brier"] else (best_log_en,m_en)

def calibrate_if_better(model,Xtr,ytr,Xte,yte):
    base=evaluate(model,Xte,yte)
    cal=CalibratedClassifierCV(model,method="isotonic",cv=5).fit(Xtr,ytr)
    calm=evaluate(cal,Xte,yte)
    return (cal,calm) if calm["Brier"]<=base["Brier"] else (model,base)

mlp_final, mlp_metrics = calibrate_if_better(best_mlp,X_tr,y_tr,X_te,y_te)

print("Final Logistic:",log_metrics,"\nFinal MLP:",mlp_metrics)

# -------- save models
joblib.dump(log_final,"model_Logistic_TUNED.joblib")
joblib.dump(mlp_final,"model_MLP_TUNED.joblib")

# -------- selected features (from selectors inside pipelines)
def selected_cols(p):
    for k in ("rfe","skb","sfm"):
        if k in p.named_steps and hasattr(p.named_steps[k],"selected_cols_"):
            return p.named_steps[k].selected_cols_
    return None

with open("selected_features.json","w") as f:
    json.dump({"rename_map":rename_map,
               "selected_features":{"Logistic":selected_cols(log_final),
                                    "MLP":selected_cols(mlp_final)}},f,indent=2)

# -------- simple ensemble (avg probs)
p_log=log_final.predict_proba(X_te)[:,1]
p_mlp=mlp_final.predict_proba(X_te)[:,1]
p_avg=0.5*p_log+0.5*p_mlp
ens={"F1":f1_score(y_te,(p_avg>=0.5).astype(int),average="weighted"),
     "ROC_AUC":roc_auc_score(y_te,p_avg),
     "Brier":brier_score_loss(y_te,p_avg)}
print("Ensemble:",ens)

Fitting 6 folds for each of 40 candidates, totalling 240 fits
Fitting 6 folds for each of 40 candidates, totalling 240 fits
Fitting 6 folds for each of 60 candidates, totalling 360 fits
L2: {'F1': 0.6306362478377882, 'ROC_AUC': np.float64(0.6829723674383869), 'Brier': np.float64(0.2209057427804729)} 
EN: {'F1': 0.6350625291976942, 'ROC_AUC': np.float64(0.6796116504854369), 'Brier': np.float64(0.2212385197949518)} 
MLP: {'F1': 0.6081135364110825, 'ROC_AUC': np.float64(0.6695917351257157), 'Brier': np.float64(0.22146414884882482)}
Final Logistic: {'F1': 0.6306362478377882, 'ROC_AUC': np.float64(0.6829723674383869), 'Brier': np.float64(0.2209057427804729)} 
Final MLP: {'F1': 0.6081135364110825, 'ROC_AUC': np.float64(0.6695917351257157), 'Brier': np.float64(0.22146414884882482)}
Ensemble: {'F1': 0.6438425990757596, 'ROC_AUC': np.float64(0.6794249439880509), 'Brier': np.float64(0.22062161449927217)}


In [14]:
# Step 1: Open Chrome and fetch NHL odds
with webdriver.Chrome() as driver:
    url = "https://betway.com/en/sports/grp/ice-hockey/north-america/nhl"
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

driver.quit()

# Step 2: Extract match odds
matches_data = []
games = soup.find_all(class_="oneLineEventItem")

for game in games:
    try:
        teams = game.find_all(class_="teamNameFirstPart")
        odds = game.find_all(class_="odds")

        if len(teams) >= 2 and len(odds) >= 2:
            matches_data.append({
                "Away Team": teams[0].text.strip(),
                "Away Odds": float(odds[0].text.strip()),
                "Home Team": teams[1].text.strip(),
                "Home Odds": float(odds[1].text.strip())
            })
    except Exception as e:
        print(f"Error extracting data: {e}")

df_odds = pd.DataFrame(matches_data)

# ---- Team-name normalization (specific to this site) ----
# 1) quick alias fixes for common sportsbook abbreviations
team_alias = {
    "LA Kings": "Los Angeles Kings",
    "Montréal Canadiens": "Montreal Canadiens",

}

def apply_alias(name: str) -> str:
    name = (name or "").strip()
    return team_alias.get(name, name)

df_odds["Home Team"] = df_odds["Home Team"].apply(apply_alias)
df_odds["Away Team"] = df_odds["Away Team"].apply(apply_alias)

# 2) optional: fuzzy match to your historical team list if df_final is available
try:
    from difflib import get_close_matches
    teams_known = set(pd.concat([df_final["Home Team"], df_final["Away Team"]]).dropna().unique())

    def normalize_team_smart(name: str) -> str:
        if name in teams_known:
            return name
        # find closest known team name if reasonably similar
        m = get_close_matches(name, list(teams_known), n=1, cutoff=0.75)
        return m[0] if m else name

    df_odds["Home Team"] = df_odds["Home Team"].apply(normalize_team_smart)
    df_odds["Away Team"] = df_odds["Away Team"].apply(normalize_team_smart)
except Exception:
    # if df_final isn't defined yet or difflib not available, just skip fuzzy step
    pass
# ---- end normalization ----

print(df_odds)

# Step 3: Automated Prediction for All Games
predictions = []

             Away Team  Away Odds             Home Team  Home Odds
0   Chicago Blackhawks       3.40      Florida Panthers       1.33
1  Pittsburgh Penguins       2.75      New York Rangers       1.50
2   Colorado Avalanche       1.90     Los Angeles Kings       1.90
3   Montreal Canadiens       2.25   Toronto Maple Leafs       1.66
4        Boston Bruins       2.60   Washington Capitals       1.52
5       Calgary Flames       2.85       Edmonton Oilers       1.44
6    Los Angeles Kings       2.60  Vegas Golden Knights       1.52


In [31]:

CSV_PATH = "odds.csv"   
MANUAL_GAMES = [
    {"Away Team": "Montreal Canadiens", "Away Odds": 2.15, "Home Team": "Toronto Maple Leafs", "Home Odds": 1.75},
    {"Away Team": "Los Angeles Kings",  "Away Odds": 2.05, "Home Team": "Edmonton Oilers",    "Home Odds": 1.90},
]


_alias = lambda x: x  

def _validate_decimal(x) -> float | None:
    try:
        v = float(str(x).replace(",", "."))
        return v if v > 1.01 else None
    except Exception:
        return None

def load_df_odds() -> pd.DataFrame:
    if os.path.exists(CSV_PATH):
        df = pd.read_csv(CSV_PATH)
    elif MANUAL_GAMES:
        df = pd.DataFrame(MANUAL_GAMES)
    else:
        raise SystemExit(f"No odds provided. Create {CSV_PATH} or fill MANUAL_GAMES list.")

    required = ["Away Team", "Away Odds", "Home Team", "Home Odds"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise SystemExit(f"Missing columns in odds input: {missing}")

    # clean
    df["Away Team"] = df["Away Team"].map(_alias)
    df["Home Team"] = df["Home Team"].map(_alias)
    df["Away Odds"] = df["Away Odds"].map(_validate_decimal)
    df["Home Odds"] = df["Home Odds"].map(_validate_decimal)

    
    bad = df[df[["Away Odds","Home Odds"]].isna().any(axis=1)]
    if not bad.empty:
        print(f"⚠️ Dropping {len(bad)} row(s) with invalid odds:")
        print(bad[["Away Team","Away Odds","Home Team","Home Odds"]])

    df = df.dropna(subset=["Away Team","Home Team","Away Odds","Home Odds"]).copy()
    return df.reset_index(drop=True)


df_odds = load_df_odds()
print("✅ Odds loaded:")
print(df_odds)


✅ Odds loaded:
            Away Team  Away Odds            Home Team  Home Odds
0  Montreal Canadiens       2.15  Toronto Maple Leafs       1.75
1   Los Angeles Kings       2.05      Edmonton Oilers       1.90


In [34]:
# ===================== CONFIG (edit) =====================
SEED = 42
MODEL_LOG_PATH = "model_Logistic_TUNED_HEAVY.joblib"
MODEL_MLP_PATH = "model_MLP_TUNED_HEAVY.joblib"
META_PATH = "selected_features_heavy.json"   # if missing, fallback map below
ENSEMBLE_WEIGHTS = (0.5, 0.5)                # (logistic, mlp)
BANKROLL = 100.0
PREDICTIONS_CSV = "predictions.csv"
# =========================================================

os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED); np.random.seed(SEED)


log_model = joblib.load(MODEL_LOG_PATH)
mlp_model = joblib.load(MODEL_MLP_PATH)


fallback_rename = {
    "Opponent Strength":"Opponent_Strength",
    "Rank Difference":"Rank_Difference",
    "Last 10 Wins":"Last_10_Wins",
    "Home Played Yesterday":"Home_Played_Yesterday",
    "Away Played Yesterday":"Away_Played_Yesterday",
    "Home Advantage":"Home_Advantage",
    "Win Streak Impact":"Win_Streak_Impact",
    "Away Rest Days Since Last Game":"Away_Rest_Days_Since_Last_Game",
    "Home Rest Days Since Last Game":"Home_Rest_Days_Since_Last_Game",
}
if os.path.exists(META_PATH):
    with open(META_PATH, "r") as f:
        rename_map = json.load(f).get("rename_map", fallback_rename)
else:
    rename_map = fallback_rename

FEATURE_COLS = list(rename_map.keys())
FEATURE_COLS_RENAMED = [rename_map[c] for c in FEATURE_COLS]
w_log, w_mlp = ENSEMBLE_WEIGHTS


def kelly_criterion(p_win: float, dec_odds: float) -> float:
    b = dec_odds - 1.0
    q = 1.0 - p_win
    return 0.0 if b <= 0 else max(0.0, (b * p_win - q) / b)

def _latest_value(df_hist, team, col_if_home, col_if_away):
    m = (df_hist["Home Team"] == team) | (df_hist["Away Team"] == team)
    last = df_hist.loc[m].iloc[-1]
    return last[col_if_home] if last["Home Team"] == team else last[col_if_away]

def build_features_for_match(home_team, away_team, df_hist, df_rank):
    r_home = int(df_rank.loc[df_rank["Team"] == home_team, "Rank"].iloc[0])
    r_away = int(df_rank.loc[df_rank["Team"] == away_team, "Rank"].iloc[0])
    return {
        "Rank Difference": r_home - r_away,
        "Last 10 Wins": (
            _latest_value(df_hist, home_team, "Home Last 10 Wins", "Away Last 10 Wins")
            - _latest_value(df_hist, away_team, "Away Last 10 Wins", "Home Last 10 Wins")
        ),
        "Home Played Yesterday": bool(_latest_value(df_hist, home_team, "Home Played Yesterday", "Away Played Yesterday")),
        "Away Played Yesterday": bool(_latest_value(df_hist, away_team, "Away Played Yesterday", "Home Played Yesterday")),
        "Home Advantage": (
            _latest_value(df_hist, home_team, "Home Win Rate", "Away Win Rate")
            - _latest_value(df_hist, away_team, "Away Win Rate", "Home Win Rate")
        ),
        "Win Streak Impact": (
            _latest_value(df_hist, home_team, "Home Team Overall Win Streak Before Game", "Away Team Overall Win Streak Before Game")
            - _latest_value(df_hist, away_team, "Away Team Overall Win Streak Before Game", "Home Team Overall Win Streak Before Game")
        ),
        "Away Rest Days Since Last Game": _latest_value(df_hist, away_team, "Away Rest Days Since Last Game", "Home Rest Days Since Last Game"),
        "Home Rest Days Since Last Game": _latest_value(df_hist, home_team, "Home Rest Days Since Last Game", "Away Rest Days Since Last Game"),
        "Opponent Strength": (
            _latest_value(df_hist, home_team, "Home Opponent Strength", "Away Opponent Strength")
            - _latest_value(df_hist, away_team, "Away Opponent Strength", "Home Opponent Strength")
        ),
    }

def to_X_row(feat_dict: dict) -> pd.DataFrame:
    row = {rename_map[k]: feat_dict[k] for k in FEATURE_COLS}
    return pd.DataFrame([row])

pred_rows, pre_bets, total_pre = [], [], 0.0

for _, r in df_odds.iterrows():
    h, a = r["Home Team"], r["Away Team"]
    oh, oa = float(r["Home Odds"]), float(r["Away Odds"])

    feats = build_features_for_match(h, a, df_final, df_rank)
    X_one = to_X_row(feats)

    p_h_log = log_model.predict_proba(X_one)[:, 1].item()
    p_h_mlp = mlp_model.predict_proba(X_one)[:, 1].item()
    p_h = w_log * p_h_log + w_mlp * p_h_mlp

    k_h = kelly_criterion(p_h, oh)
    k_a = kelly_criterion(1.0 - p_h, oa)

    bh_pre = k_h * BANKROLL
    ba_pre = k_a * BANKROLL
    pre_bets.append((bh_pre, ba_pre))
    total_pre += bh_pre + ba_pre

    pred_rows.append({
        "Date": datetime.now().strftime("%Y-%m-%d"),
        "Home Team": h, "Away Team": a,
        "Home Odds": oh, "Away Odds": oa,
        "p_home_ens": round(p_h, 4)
    })


for i, row in enumerate(pred_rows):
    bh, ba = pre_bets[i]
    if total_pre > BANKROLL > 0:
        s = BANKROLL / total_pre
        bh, ba = bh * s, ba * s
        row["Scaled"] = True
    else:
        row["Scaled"] = False

    # Only bet if Kelly > 0; round to nearest $1
    row["Home Bet"] = round(bh) if bh > 0 else 0
    row["Away Bet"] = round(ba) if ba > 0 else 0

pred_df = pd.DataFrame(pred_rows)

print("✅ Predictions:")
print(pred_df[["Date","Away Team","Away Odds","Home Team","Home Odds","p_home_ens","Home Bet","Away Bet","Scaled"]])

#  append to CSV
if os.path.exists(PREDICTIONS_CSV):
    old = pd.read_csv(PREDICTIONS_CSV)
    pd.concat([old, pred_df], ignore_index=True).to_csv(PREDICTIONS_CSV, index=False)
else:
    pred_df.to_csv(PREDICTIONS_CSV, index=False)


✅ Predictions:
         Date           Away Team  Away Odds            Home Team  Home Odds  \
0  2025-10-05  Montreal Canadiens       2.15  Toronto Maple Leafs       1.75   
1  2025-10-05   Los Angeles Kings       2.05      Edmonton Oilers       1.90   

   p_home_ens  Home Bet  Away Bet  Scaled  
0      0.5819         2         0   False  
1      0.5361         2         0   False  
